In [1]:
import time
import pandas as pd
import psycopg2
import datetime
from tqdm import trange
import matplotlib.pyplot as plt 
import matplotlib as mpl 
import seaborn as sns
import math
# %matplotlib inline 
import platform
import numpy as np
from matplotlib import font_manager, rc # plt.rcParams['axes.unicode_minus'] = False 
if platform.system() == 'Darwin': rc('font', family='AppleGothic') 
elif platform.system() == 'Windows': 
    path = "c:/Windows/Fonts/malgun.ttf" 
    font_name = font_manager.FontProperties(fname=path).get_name() 
    rc('font', family=font_name) 
else: 
    print('Unknown system... sorry~~~~')
plt.rcParams["figure.figsize"] = (14,4)
plt.rcParams['lines.linewidth'] = 2
from tqdm import tqdm
from tqdm import trange
from datetime import datetime, timedelta


In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))


In [3]:
# 시계열
from datetime import datetime as dt
from statsmodels.tsa.arima_model import ARIMA
from dateutil.relativedelta import relativedelta
from statsmodels.tsa.api import SimpleExpSmoothing, Holt, ExponentialSmoothing

In [4]:
    # row 생략 없이 출력
pd.set_option('display.max_rows', 1000)
# col 생략 없이 출력
pd.set_option('display.max_columns', 1000)
plt.rcParams["figure.figsize"] = (14,4)
plt.rcParams['lines.linewidth'] = 2
pd.options.display.float_format = '{:.5f}'.format

In [136]:
# DB 연결 부분
uswer = 'dt.intern'
host = '10.80.0.234'
port = '5439'
dbname = 'fnf'
con = psycopg2.connect(dbname='fnf',host='10.80.0.234', port='5439', user = 'dt.intern', password='DTintn0926')
cur = con.cursor()

In [23]:
# 전체 매장 입출판재
total_shop_stock = pd.read_csv("total_shop_stock.csv")
total_shop_stock = total_shop_stock.iloc[ :,1:]

# 매장정보
shop_info_pd = pd.read_csv("shop_info_pd.csv")
shop_info_pd = shop_info_pd.iloc[ :,1:]

# bucket_dwalker 관련 매장입출판재(2019)
bucket_dwalker = pd.read_csv('bucket_dwalker.csv')
bucket_dwalker = bucket_dwalker.iloc[ :,1:]

# bucket_dwalker_v2 관련 매장입출판재
bucket_dwalker_v2 = pd.read_csv('bucket_dwalker_v2.csv')
bucket_dwalker_v2 = bucket_dwalker_v2.iloc[ :,1:]

# 아울렛 상설을 제외한 매장키 
shop_list_pd = pd.read_csv('shop_list.csv')
shop_list_pd = shop_list_pd.iloc[ :,1:]

# 전체 매장 비어있는 주차 채운 데이터 (버킷 디워커 기준)
shop_weeks_buket_df = pd.read_csv('shop_weeks_buket.csv')
shop_weeks_buket_df = shop_weeks_buket_df.iloc[ :,1:]

# 전체 매장 비어있는 주차 채운 데이터 (버킷 디워커 v2 기준)
shop_weeks_buket_v2_df = pd.read_csv('shop_weeks_buket_v2.csv')
shop_weeks_buket_v2_df = shop_weeks_buket_v2_df.iloc[ :,1:]

# 전체 매장 비어있는것 채워넣은 입출판재 합쳐 놓은 것
shop_total_sales = pd.read_csv("shop_total_sales.csv")
shop_total_sales = shop_total_sales.iloc[ :,1:]

# 버킷디워커 물류 재고량
bk_stock_scsweek_pd = pd.read_csv("bk_stock_scsweek_pd.csv")
bk_stock_scsweek_pd = bk_stock_scsweek_pd.iloc[ :,1:]

# 버킷디워커 물류 재고량
v2_stock_scsweek_pd = pd.read_csv("v2_stock_scsweek_pd.csv")
v2_stock_scsweek_pd = v2_stock_scsweek_pd.iloc[ :,1:]

In [6]:
# DB 연결 부분
uswer = 'dt.intern'
host = '10.80.0.234'
port = '5439'
dbname = 'fnf'
con = psycopg2.connect(dbname='fnf',host='10.80.0.234', port='5439', user = 'dt.intern', password='DTintn0926')
cur = con.cursor()

In [11]:
def mae(prediction, correct):
    prediction = np.array(prediction)
    correct = np.array(correct)
    
    difference = correct - prediction
    abs_val = abs(difference)
    
    score = abs_val.mean()
    
    return score

# mae_scorer = make_scorer(mae)
# mae_scorer

In [12]:
def make_ewm_arr(data_frame, span_num):
    arr_ewm = np.array([])
#     for i in data_frame.store_id.unique():
    df_set = data_frame
        # 여기에서 지정하는 span값은 위 수식에서 N에 해당한다.
    arr_ewm = np.concatenate((arr_ewm, df_set.week_sale_qty.ewm(span=span_num).mean().values) )
    
    return arr_ewm

In [137]:
# 2019년 버킷디워커 물류 수량
start = time.time()

bk_stock_scsweek = '''

SELECT weekkey ,scskey, week_accept_qty ,week_wsl_takeout_qty, wms_stock_qty ,stock_qty,domestic_sale_qty
FROM ax.vw_f_scsweek vfs 
WHERE scskey  IN ('X19SDXSH09911BE230','X19SDXSH09911BE235','X19SDXSH09911BE240','X19SDXSH09911BE260','X19SDXSH09911BE265','X19SDXSH09911BE270',
'X19SDXSH09911GR230','X19SDXSH09911GR235','X19SDXSH09911GR240','X19SDXSH09911GR260','X19SDXSH09911GR265','X19SDXSH09911GR270',
'X19SDXSH09911WH230','X19SDXSH09911WH235','X19SDXSH09911WH240','X19SDXSH09911WH260','X19SDXSH09911WH265','X19SDXSH09911WH270',
'X19SDXSH09911BK230','X19SDXSH09911BK235','X19SDXSH09911BK240','X19SDXSH09911BK260','X19SDXSH09911BK265','X19SDXSH09911BK270') 
AND weekkey BETWEEN To_date('2018-12-31', 'YYYY-MM-DD') AND To_date('2019-12-31', 'YYYY-MM-DD')


'''


cur.execute(bk_stock_scsweek)
bk_stock_scsweek = cur.fetchall()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

bk_stock_scsweek_pd = pd.DataFrame(bk_stock_scsweek)
bk_stock_scsweek_pd.columns = ['weekkey' ,'scskey', 'week_accept_qty' ,'week_wsl_takeout_qty', 'wms_stock_qty' ,'stock_qty','domestic_sale_qty']

time : 6.416370391845703


In [138]:
# 2020년 v2 버킷디워커 물류 수량
start = time.time()

v2_stock_scsweek = '''

SELECT weekkey ,scskey, week_accept_qty ,week_wsl_takeout_qty, wms_stock_qty ,stock_qty,domestic_sale_qty
FROM ax.vw_f_scsweek vfs 
WHERE scskey  IN ('X20SDXSHA5011OW230','X20SDXSHA5011OW235','X20SDXSHA5011OW240','X20SDXSHA5011OW260','X20SDXSHA5011OW265','X20SDXSHA5011OW270',
'X20SDXSHA1011WH230','X20SDXSHA1011WH235','X20SDXSHA1011WH240','X20SDXSHA1011WH260','X20SDXSHA1011WH265','X20SDXSHA1011WH270',
'X20SDXSHA3011BE230','X20SDXSHA3011BE235','X20SDXSHA3011BE240','X20SDXSHA3011BE260','X20SDXSHA3011BE265','X20SDXSHA3011BE270'
) AND weekkey BETWEEN To_date('2019-11-18', 'YYYY-MM-DD') AND To_date('2020-12-31', 'YYYY-MM-DD')


'''


cur.execute(v2_stock_scsweek)
v2_stock_scsweek = cur.fetchall()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

v2_stock_scsweek_pd = pd.DataFrame(v2_stock_scsweek)
v2_stock_scsweek_pd.columns = ['weekkey' ,'scskey', 'week_accept_qty' ,'week_wsl_takeout_qty', 'wms_stock_qty' ,'stock_qty','domestic_sale_qty']

time : 0.14747023582458496


In [195]:
v2_stock_scsweek_pd

,weekkey,scskey,week_accept_qty,week_wsl_takeout_qty,wms_stock_qty,stock_qty,domestic_sale_qty
0,2020-02-10,X20SDXSHA1011WH265,0,0,1048,1587,944
1,2020-04-27,X20SDXSHA1011WH265,50,1,63,929,2241
2,2020-05-18,X20SDXSHA1011WH265,71,0,198,977,2552
3,2020-06-01,X20SDXSHA1011WH265,314,0,459,1183,2720
4,2020-06-15,X20SDXSHA1011WH265,572,0,1198,1886,2859
...,...,...,...,...,...,...,...
1057,2020-08-17,X20SDXSHA5011OW235,0,0,2635,3695,4776
1058,2020-08-31,X20SDXSHA5011OW235,0,0,2574,3608,4862
1059,2020-10-19,X20SDXSHA5011OW235,0,1,1990,2972,5493
1060,2020-11-16,X20SDXSHA5011OW235,0,0,1476,2457,6007


# 재고가 0 인 지점

## 버킷 디워커 재고량 0 인 지점

In [158]:
shop_weeks_buket_df_dep = shop_weeks_buket_df[shop_weeks_buket_df['type_zone_nm']=='백화점']
bk_shopkey_unique =  shop_weeks_buket_df_dep.shopkey.unique()
bk_scskey_unique =  shop_weeks_buket_df_dep.scskey.unique()
shop_weeks_buket_df_dep = shop_weeks_buket_df_dep[shop_weeks_buket_df_dep['shopkey'] != 'X10047']

bk_shop_qty = pd.DataFrame()

for i in bk_shopkey_unique:
    uniqe_shop = shop_weeks_buket_df_dep[shop_weeks_buket_df_dep['shopkey']==i]
    for j in bk_scskey_unique:
        uniqe_shop_scskey = uniqe_shop[uniqe_shop['scskey']==j]
        ewm_value = make_ewm_arr(uniqe_shop_scskey, 2)
        uniqe_shop_scskey.insert(11, 'amount_2ewm', ewm_value)
        ewm_value = np.append([0],ewm_value,axis=0)
        uniqe_shop_scskey['amount_2ewm_week'] = ewm_value[:-1]
        for i in range(0,len(ewm_value)):
#             ewm_value[i] = round(ewm_value[i] , 0)+1
            ewm_value[i] = math.ceil(ewm_value[i])+1

        uniqe_shop_scskey['qty'] = ewm_value[:-1]
        bk_shop_qty = pd.concat([bk_shop_qty, uniqe_shop_scskey], axis=0)

bk_zero = bk_shop_qty[(bk_shop_qty['stock_qty']==0) & (bk_shop_qty['takeout_qty']>0)]


<ipython-input-158-c2e77f69cd77>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniqe_shop_scskey['amount_2ewm_week'] = ewm_value[:-1]
<ipython-input-158-c2e77f69cd77>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniqe_shop_scskey['qty'] = ewm_value[:-1]


In [159]:
## 버킷디워커 재고량 0 인 지점과 실 재고량 확인 

In [160]:
bk_zero_point = bk_zero[bk_zero.qty - bk_zero.week_sale_qty>0]
bk_zero_point = bk_zero_point.sort_values(['weekkey','scskey'])

In [161]:
bk_stock_scsweek_pd['weekkey'] = pd.to_datetime(bk_stock_scsweek_pd['weekkey'])
bk_zero_point['weekkey'] = pd.to_datetime(bk_zero_point['weekkey'])

In [102]:
def shop_weeks_buket_function(shop_weeks_buket_df):
    shop_weeks_buket_df_dep = shop_weeks_buket_df[shop_weeks_buket_df['type_zone_nm']=='백화점']
    bk_shopkey_unique =  shop_weeks_buket_df_dep.shopkey.unique()
    bk_scskey_unique =  shop_weeks_buket_df_dep.scskey.unique()
    shop_weeks_buket_df_dep = shop_weeks_buket_df_dep[shop_weeks_buket_df_dep['shopkey'] != 'X10047']

    bk_shop_qty = pd.DataFrame()

    for i in bk_shopkey_unique:
        uniqe_shop = shop_weeks_buket_df_dep[shop_weeks_buket_df_dep['shopkey']==i]
        for j in bk_scskey_unique:
            uniqe_shop_scskey = uniqe_shop[uniqe_shop['scskey']==j]
            ewm_value = make_ewm_arr(uniqe_shop_scskey, 2)
            uniqe_shop_scskey.insert(11, 'amount_2ewm', ewm_value)
            ewm_value = np.append([0],ewm_value,axis=0)
            uniqe_shop_scskey['amount_2ewm_week'] = ewm_value[:-1]
            for i in range(0,len(ewm_value)):
    #             ewm_value[i] = round(ewm_value[i] , 0)+1
                ewm_value[i] = math.ceil(ewm_value[i])+1

            uniqe_shop_scskey['qty'] = ewm_value[:-1]
            bk_shop_qty = pd.concat([bk_shop_qty, uniqe_shop_scskey], axis=0)

    bk_zero = bk_shop_qty[(bk_shop_qty['stock_qty']==0) & (bk_shop_qty['takeout_qty']>0)]
    return bk_zero


## 버킷 디워커 창고 재고량 있는지 확인

In [162]:

bk_zero_point = bk_zero[bk_zero.qty - bk_zero.week_sale_qty>0]
bk_zero_point = bk_zero_point.sort_values(['weekkey','scskey'])
bk_stock_scsweek_pd['weekkey'] = pd.to_datetime(bk_stock_scsweek_pd['weekkey'])
bk_zero_point['weekkey'] = pd.to_datetime(bk_zero_point['weekkey'])

# 실 재고량과 비교 
bk_true_zero_count = 0
for i in bk_zero_point.weekkey.unique(): # 날짜 
    for j in bk_zero_point.scskey.unique(): # scskey
        bk_zero_point_check = bk_zero_point[(bk_zero_point['weekkey']==i) & (bk_zero_point['scskey']==j)] # 해당 날짜의 scskey가 있는지 확인 
        if len(bk_zero_point_check) == 0 : # 없으면 다음으로 넘기고 
            break
            
        # 있을 경우에 해당 날짜 scskey 의 창고 물량을 확인
        wms_stock = bk_stock_scsweek_pd[(bk_stock_scsweek_pd['weekkey']==bk_zero_point_check.iloc[0].weekkey) & (bk_stock_scsweek_pd.scskey ==bk_zero_point_check.iloc[0].scskey) ][['wms_stock_qty']].iloc[0][0] 

        for z in range(0,len(bk_zero_point_check)):            # 창고내 해당 물건의 재고량 - 재고 0 이였던 지점의 qty 
            if wms_stock - bk_zero_point_check.iloc[z].qty>0:  # 1보다  클 경우만 
                bk_true_zero_count+=1                          # count +1 하고
                wms_stock-= bk_zero_point_check.iloc[z].qty    # wms의 값 또한 빼기한다.
            else:                                              # 1보다 작으면 break 하고 다음 for 문으로 넘긴다.
                break

In [163]:
# 재고 반영
bk_true_zero_count

1309

In [164]:
# 재고 없이 로직으로
len(bk_zero_point)

1863

In [165]:
# 재고가 0 인 지점 
len(bk_zero)

2225

In [179]:
# 실 재고로 비교하는 함수
def bk_stock_zero_function(bk_zero):
    bk_zero_point = bk_zero[bk_zero.qty - bk_zero.week_sale_qty>0]
    bk_zero_point = bk_zero_point.sort_values(['weekkey','scskey'])
    bk_stock_scsweek_pd['weekkey'] = pd.to_datetime(bk_stock_scsweek_pd['weekkey'])
    bk_zero_point['weekkey'] = pd.to_datetime(bk_zero_point['weekkey'])

    # 실 재고량과 비교 
    bk_true_zero_count = 0
    for i in bk_zero_point.weekkey.unique(): # 날짜 
        for j in bk_zero_point.scskey.unique(): # scskey
            bk_zero_point_check = bk_zero_point[(bk_zero_point['weekkey']==i) & (bk_zero_point['scskey']==j)] # 해당 날짜의 scskey가 있는지 확인 
            if len(bk_zero_point_check) == 0 : # 없으면 다음으로 넘기고 
                break

            # 있을 경우에 해당 날짜 scskey 의 창고 물량을 확인
            wms_stock = bk_stock_scsweek_pd[(bk_stock_scsweek_pd['weekkey']==bk_zero_point_check.iloc[0].weekkey) & (bk_stock_scsweek_pd.scskey ==bk_zero_point_check.iloc[0].scskey) ][['wms_stock_qty']].iloc[0][0] 

            for z in range(0,len(bk_zero_point_check)):            # 창고내 해당 물건의 재고량 - 재고 0 이였던 지점의 qty 
                if wms_stock - bk_zero_point_check.iloc[z].qty>0:  # 1보다  클 경우만 
                    bk_true_zero_count+=1                          # count +1 하고
                    wms_stock-= bk_zero_point_check.iloc[z].qty    # wms의 값 또한 빼기한다.
                else:                                              # 1보다 작으면 break 하고 다음 for 문으로 넘긴다.
                    break
    zero_count = len(bk_zero) - bk_true_zero_count
    return zero_count

## v2 재고량 0인 지점 확인

In [14]:
shop_weeks_buket_v2_df_dep = shop_weeks_buket_v2_df[shop_weeks_buket_v2_df['type_zone_nm']=='백화점']
shop_weeks_buket_v2_df_dep = shop_weeks_buket_v2_df_dep[shop_weeks_buket_v2_df_dep['shopkey'] != 'X10047']
v2_shopkey_unique =  shop_weeks_buket_v2_df_dep.shopkey.unique()
v2_scskey_unique =  shop_weeks_buket_v2_df_dep.scskey.unique()

v2_shop_qty = pd.DataFrame()

for i in v2_shopkey_unique:
    uniqe_shop = shop_weeks_buket_v2_df_dep[shop_weeks_buket_v2_df_dep['shopkey']==i]
    for j in v2_scskey_unique:
        uniqe_shop_scskey = uniqe_shop[uniqe_shop['scskey']==j]
        ewm_value = make_ewm_arr(uniqe_shop_scskey, 2)
        uniqe_shop_scskey.insert(11, 'amount_2ewm', ewm_value)
        ewm_value = np.append([0],ewm_value,axis=0)
        uniqe_shop_scskey['amount_2ewm_week'] = ewm_value[:-1]
        for i in range(0,len(ewm_value)):
#             ewm_value[i] = round(ewm_value[i] , 0)+1
            ewm_value[i] = math.ceil(ewm_value[i])+1

        uniqe_shop_scskey['qty'] = ewm_value[:-1]
        v2_shop_qty = pd.concat([v2_shop_qty, uniqe_shop_scskey], axis=0)
v2_zero = v2_shop_qty[(v2_shop_qty['stock_qty']==0) & (v2_shop_qty['takeout_qty']>0)]

<ipython-input-14-7f246e1b3243>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniqe_shop_scskey['amount_2ewm_week'] = ewm_value[:-1]
<ipython-input-14-7f246e1b3243>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniqe_shop_scskey['qty'] = ewm_value[:-1]


In [ ]:
def shop_weeks_buket_function(shop_weeks_buket_df):
    shop_weeks_buket_df_dep = shop_weeks_buket_df[shop_weeks_buket_df['type_zone_nm']=='백화점']
    bk_shopkey_unique =  shop_weeks_buket_df_dep.shopkey.unique()
    bk_scskey_unique =  shop_weeks_buket_df_dep.scskey.unique()
    shop_weeks_buket_df_dep = shop_weeks_buket_df_dep[shop_weeks_buket_df_dep['shopkey'] != 'X10047']

    bk_shop_qty = pd.DataFrame()

    for i in bk_shopkey_unique:
        uniqe_shop = shop_weeks_buket_df_dep[shop_weeks_buket_df_dep['shopkey']==i]
        for j in bk_scskey_unique:
            uniqe_shop_scskey = uniqe_shop[uniqe_shop['scskey']==j]
            ewm_value = make_ewm_arr(uniqe_shop_scskey, 2)
            uniqe_shop_scskey.insert(11, 'amount_2ewm', ewm_value)
            ewm_value = np.append([0],ewm_value,axis=0)
            uniqe_shop_scskey['amount_2ewm_week'] = ewm_value[:-1]
            for i in range(0,len(ewm_value)):
    #             ewm_value[i] = round(ewm_value[i] , 0)+1
                ewm_value[i] = math.ceil(ewm_value[i])+1

            uniqe_shop_scskey['qty'] = ewm_value[:-1]
            bk_shop_qty = pd.concat([bk_shop_qty, uniqe_shop_scskey], axis=0)

    bk_zero = bk_shop_qty[(bk_shop_qty['stock_qty']==0) & (bk_shop_qty['takeout_qty']>0)]
    return bk_zero


In [212]:
def shop_weeks_buket_v2_function(shop_weeks_buket_v2_df):
    shop_weeks_buket_v2_df_dep = shop_weeks_buket_v2_df[shop_weeks_buket_v2_df['type_zone_nm']=='백화점']
    shop_weeks_buket_v2_df_dep = shop_weeks_buket_v2_df_dep[shop_weeks_buket_v2_df_dep['shopkey'] != 'X10047']
    v2_shopkey_unique =  shop_weeks_buket_v2_df_dep.shopkey.unique()
    v2_scskey_unique =  shop_weeks_buket_v2_df_dep.scskey.unique()

    v2_shop_qty = pd.DataFrame()

    for i in v2_shopkey_unique:
        uniqe_shop = shop_weeks_buket_v2_df_dep[shop_weeks_buket_v2_df_dep['shopkey']==i]
        for j in v2_scskey_unique:
            uniqe_shop_scskey = uniqe_shop[uniqe_shop['scskey']==j]
            ewm_value = make_ewm_arr(uniqe_shop_scskey, 2)
            uniqe_shop_scskey.insert(11, 'amount_2ewm', ewm_value)
            ewm_value = np.append([0],ewm_value,axis=0)
            uniqe_shop_scskey['amount_2ewm_week'] = ewm_value[:-1]
            for i in range(0,len(ewm_value)):
    #             ewm_value[i] = round(ewm_value[i] , 0)+1
                ewm_value[i] = math.ceil(ewm_value[i])+1

            uniqe_shop_scskey['qty'] = ewm_value[:-1]
            v2_shop_qty = pd.concat([v2_shop_qty, uniqe_shop_scskey], axis=0)
    v2_zero = v2_shop_qty[(v2_shop_qty['stock_qty']==0) & (v2_shop_qty['takeout_qty']>0)]
    return v2_zero


## v2 버킷 디워커 창고 재고량 있는지 확인

In [ ]:
## 버킷 디워커 v2  창고 재고량 있는지 확인

v2_zero_point = v2_zero[v2_zero.qty - v2_zero.week_sale_qty>0]
v2_zero_point = v2_zero_point.sort_values(['weekkey','scskey'])

In [35]:
v2_stock_scsweek_pd['weekkey'] = pd.to_datetime(v2_stock_scsweek_pd['weekkey'])
v2_zero_point['weekkey'] = pd.to_datetime(v2_zero_point['weekkey'])

In [36]:
# 실 재고량과 비교 
v2_true_zero_count = 0
for i in v2_zero_point.weekkey.unique(): # 날짜 
    for j in v2_zero_point.scskey.unique(): # scskey
        v2_zero_point_check = v2_zero_point[(v2_zero_point['weekkey']==i) & (v2_zero_point['scskey']==j)] # 해당 날짜의 scskey가 있는지 확인 
        if len(v2_zero_point_check) == 0 : # 없으면 다음으로 넘기고 
            break
            
        # 있을 경우에 해당 날짜 scskey 의 창고 물량을 확인
        v2_wms_stock = v2_stock_scsweek_pd[(v2_stock_scsweek_pd['weekkey']==v2_zero_point_check.iloc[0].weekkey) & (v2_stock_scsweek_pd.scskey ==v2_zero_point_check.iloc[0].scskey) ][['wms_stock_qty']].iloc[0][0] 

        for z in range(0,len(v2_zero_point_check)):            # 창고내 해당 물건의 재고량 - 재고 0 이였던 지점의 qty 
            if v2_wms_stock - v2_zero_point_check.iloc[z].qty>0:  # 1보다  클 경우만 
                v2_true_zero_count+=1                          # count +1 하고
                v2_wms_stock-= v2_zero_point_check.iloc[z].qty    # wms의 값 또한 빼기한다.
            else:                                              # 1보다 작으면 break 하고 다음 for 문으로 넘긴다.
                break

In [37]:
v2_true_zero_count

50

In [38]:
len(v2_zero_point)

78

In [39]:
len(v2_zero)

240

In [180]:
# 실 재고로 비교하는 함수
def v2_stock_zero_function(v2_zero):
    v2_zero_point = v2_zero[v2_zero.qty - v2_zero.week_sale_qty>0]
    v2_zero_point = v2_zero_point.sort_values(['weekkey','scskey'])
    v2_stock_scsweek_pd['weekkey'] = pd.to_datetime(v2_stock_scsweek_pd['weekkey'])
    v2_zero_point['weekkey'] = pd.to_datetime(v2_zero_point['weekkey'])

        # 실 재고량과 비교 
    v2_true_zero_count = 0
    for i in v2_zero_point.weekkey.unique(): # 날짜 
        for j in v2_zero_point.scskey.unique(): # scskey
            v2_zero_point_check = v2_zero_point[(v2_zero_point['weekkey']==i) & (v2_zero_point['scskey']==j)] # 해당 날짜의 scskey가 있는지 확인 
            if len(v2_zero_point_check) == 0 : # 없으면 다음으로 넘기고 
                break

            # 있을 경우에 해당 날짜 scskey 의 창고 물량을 확인
            v2_wms_stock = v2_stock_scsweek_pd[(v2_stock_scsweek_pd['weekkey']==v2_zero_point_check.iloc[0].weekkey) & (v2_stock_scsweek_pd.scskey ==v2_zero_point_check.iloc[0].scskey) ][['wms_stock_qty']].iloc[0][0] 

            for z in range(0,len(v2_zero_point_check)):            # 창고내 해당 물건의 재고량 - 재고 0 이였던 지점의 qty 
                if v2_wms_stock - v2_zero_point_check.iloc[z].qty>0:  # 1보다  클 경우만 
                    v2_true_zero_count+=1                          # count +1 하고
                    v2_wms_stock-= v2_zero_point_check.iloc[z].qty    # wms의 값 또한 빼기한다.
                else:                                              # 1보다 작으면 break 하고 다음 for 문으로 넘긴다.
                    break
    zero_count = len(v2_zero) - v2_true_zero_count
    return zero_count

# 과잉 재고가 되지 않는지 (MAE 값 비교)

In [ ]:
## 버킷 디워커 MAE 값 비교

In [63]:
bk_shopkey_unique = bk_shop_qty.shopkey.unique()
bk_scskey_unique = bk_shop_qty.scskey.unique()
bk_ori_mae_list=[]
bk_imp_mae_list=[]

bk_score_list = []
bk_score_list2 = []

for i in bk_shopkey_unique:
    for j in bk_scskey_unique:
#         print(test1[(test1['shopkey']==i) & (test1['scskey']==j)].week_takeout_qty[1:])
        bk_ori_mae_list.append(mae( bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].week_takeout_qty[1:] ,bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].stock_qty[:-1] ))
        bk_imp_mae_list.append(mae( bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].week_takeout_qty[1:-1] ,bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].qty[1:-1] ))

        bk_score_list.append ( sum((bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].week_takeout_qty[1:]-bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].stock_qty[:-1] ).dropna()))
        bk_score_list2.append ( sum((bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].week_takeout_qty[1:]-bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].qty[1:] ).dropna()))

#         break
#     break
        
    

In [64]:
np.mean(bk_ori_mae_list)

4.663024475524476

In [65]:
np.mean(bk_imp_mae_list)

1.9543969102792633

In [55]:
## v2 버킷 디워커 MAE 값 비교

In [51]:
v2_shopkey_unique = v2_shop_qty.shopkey.unique()
v2_scskey_unique = v2_shop_qty.scskey.unique()
v2_ori_mae_list=[]
v2_imp_mae_list=[]

v2_score_list = []
v2_score_list2 = []

for i in v2_shopkey_unique:
    for j in v2_scskey_unique:
#         print(test1[(test1['shopkey']==i) & (test1['scskey']==j)].week_takeout_qty[1:])
        v2_ori_mae_list.append(mae( v2_shop_qty[(v2_shop_qty['shopkey']==i) & (v2_shop_qty['scskey']==j)].week_takeout_qty[1:] ,v2_shop_qty[(v2_shop_qty['shopkey']==i) & (v2_shop_qty['scskey']==j)].stock_qty[:-1] ))
        v2_imp_mae_list.append(mae( v2_shop_qty[(v2_shop_qty['shopkey']==i) & (v2_shop_qty['scskey']==j)].week_takeout_qty[1:-1] ,v2_shop_qty[(v2_shop_qty['shopkey']==i) & (v2_shop_qty['scskey']==j)].qty[1:-1] ))

        v2_score_list.append ( sum((v2_shop_qty[(v2_shop_qty['shopkey']==i) & (v2_shop_qty['scskey']==j)].week_takeout_qty[1:]-v2_shop_qty[(v2_shop_qty['shopkey']==i) & (v2_shop_qty['scskey']==j)].stock_qty[:-1] ).dropna()))
        v2_score_list2.append ( sum((v2_shop_qty[(v2_shop_qty['shopkey']==i) & (v2_shop_qty['scskey']==j)].week_takeout_qty[1:]-v2_shop_qty[(v2_shop_qty['shopkey']==i) & (v2_shop_qty['scskey']==j)].qty[1:] ).dropna()))

#         break
#     break
        
    

In [53]:
np.mean(v2_ori_mae_list)

4.0462382445141065

In [54]:
np.mean(v2_imp_mae_list)

1.709418748892433

In [ ]:
## 재고 총 개수 

In [72]:
bk_shop_qty.stock_qty.sum()

95504

In [73]:
bk_shop_qty.qty.sum()

47090.0

In [67]:
v2_shop_qty.stock_qty.sum()

103277

In [69]:
v2_shop_qty.qty.sum()

51866.0

# 매장별 부족한 입출판재 채우는 로직 

In [78]:
df_INNER_JOIN['weekkey'] = pd.to_datetime(df_INNER_JOIN['weekkey'])

In [80]:
bucket_dwalker = df_INNER_JOIN[(df_INNER_JOIN['partkey'].str.contains("X19SDXSH09911")) & (df_INNER_JOIN['weekkey'] < '2019-12-31' ) ]

In [81]:
bucket_dwalker_v2 = df_INNER_JOIN[df_INNER_JOIN['partkey'].str.contains("X20SDXSHA5011|X20SDXSHA3011|X20SDXSHA1011|X20SDXSHA1011") & (df_INNER_JOIN['weekkey'] < '2020-12-31' )]

In [82]:
# 빠져있는 주차 채워주는 로직

def zero_fill(shopkey, base, scskey_list,item):
    total=[]
    for col in base.columns:
        base[col].values[:] = 0
    for scskey in scskey_list:
        for shop in shopkey:
            frame = base.reset_index()
            target = item[(item['scskey']==scskey) & (item['shopkey']==shop )].groupby('weekkey').sum().reset_index()
            target = target.reset_index()

            for i in range(0,len(target)):
                for j in range(0,len(frame)):
                        if target.iloc[i].weekkey == frame.iloc[j].weekkey:
                            frame.iloc[j] = target.iloc[i]
                            break
            for i in range(1,len(frame)):
                if frame.iloc[i].takeout_qty ==0 and frame.iloc[i].sale_qty==0:
                    frame.iloc[i] = [ frame.iloc[i]['weekkey'], 0, frame.iloc[i-1]['takeout_qty'], 0, frame.iloc[i-1]['sale_qty'], frame.iloc[i-1]['stock_qty']]
            frame['shopkey'] =shop
            frame['ratio'] = 0
            for i in range(0,len(frame)):
                total_sale_qty= frame['week_sale_qty'].sum()
                a = frame.iloc[i]
                if a['sale_qty']==0 :
                    frame.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] , 0]
                else:
                    if a['sale_qty'] / total_sale_qty > 1:
                        frame.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] , 1]
                    else:
                        frame.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] , a['sale_qty'] / total_sale_qty]
            else:
                frame['scskey']=scskey
            frame = pd.merge(frame, shop_info_pd, left_on='shopkey', right_on='shopkey', how='inner')
            total.append(frame)
    return total


In [ ]:
## 버킷 디워커

In [86]:
# BE
scskey_list = ['X19SDXSH09911BE230','X19SDXSH09911BE235','X19SDXSH09911BE240','X19SDXSH09911BE260','X19SDXSH09911BE265','X19SDXSH09911BE270']
frame = bucket_dwalker[(bucket_dwalker['scskey']=='X19SDXSH09911BE240')][['shopkey','weekkey','week_takeout_qty','takeout_qty','week_sale_qty','sale_qty','stock_qty']].groupby('weekkey').sum()
bucket_dwalker_shop_key = bucket_dwalker['shopkey'].unique()


shop_weeks_buket = zero_fill(bucket_dwalker_shop_key, frame,scskey_list,bucket_dwalker)
be_shop_weeks_buket_df = pd.concat(shop_weeks_buket)

In [90]:
# GR
scskey_list = ['X19SDXSH09911GR230','X19SDXSH09911GR235','X19SDXSH09911GR240','X19SDXSH09911GR260','X19SDXSH09911GR265','X19SDXSH09911GR270']
frame = bucket_dwalker[(bucket_dwalker['scskey']=='X19SDXSH09911GR240')][['shopkey','weekkey','week_takeout_qty','takeout_qty','week_sale_qty','sale_qty','stock_qty']].groupby('weekkey').sum()
bucket_dwalker_shop_key = bucket_dwalker['shopkey'].unique()


shop_weeks_buket = zero_fill(bucket_dwalker_shop_key, frame,scskey_list,bucket_dwalker)
gr_shop_weeks_buket_df = pd.concat(shop_weeks_buket)

<ipython-input-82-b30433df00a6>:29: RuntimeWarning: divide by zero encountered in longlong_scalars
  if a['sale_qty'] / total_sale_qty > 1:
<ipython-input-82-b30433df00a6>:32: RuntimeWarning: divide by zero encountered in longlong_scalars
  frame.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] , a['sale_qty'] / total_sale_qty]


In [93]:
# WH
scskey_list = ['X19SDXSH09911WH230','X19SDXSH09911WH235','X19SDXSH09911WH240','X19SDXSH09911WH260','X19SDXSH09911WH265','X19SDXSH09911WH270']
frame = bucket_dwalker[(bucket_dwalker['scskey']=='X19SDXSH09911WH240')][['shopkey','weekkey','week_takeout_qty','takeout_qty','week_sale_qty','sale_qty','stock_qty']].groupby('weekkey').sum()
bucket_dwalker_shop_key = bucket_dwalker['shopkey'].unique()


shop_weeks_buket = zero_fill(bucket_dwalker_shop_key, frame,scskey_list,bucket_dwalker)
wh_shop_weeks_buket_df = pd.concat(shop_weeks_buket)

<ipython-input-82-b30433df00a6>:29: RuntimeWarning: divide by zero encountered in longlong_scalars
  if a['sale_qty'] / total_sale_qty > 1:
<ipython-input-82-b30433df00a6>:32: RuntimeWarning: divide by zero encountered in longlong_scalars
  frame.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] , a['sale_qty'] / total_sale_qty]


In [95]:
# BK

scskey_list = ['X19SDXSH09911BK230','X19SDXSH09911BK235','X19SDXSH09911BK240','X19SDXSH09911BK260','X19SDXSH09911BK265','X19SDXSH09911BK270']
frame = bucket_dwalker[(bucket_dwalker['scskey']=='X19SDXSH09911BK240')][['shopkey','weekkey','week_takeout_qty','takeout_qty','week_sale_qty','sale_qty','stock_qty']].groupby('weekkey').sum()
bucket_dwalker_shop_key = bucket_dwalker['shopkey'].unique()


shop_weeks_buket = zero_fill(bucket_dwalker_shop_key, frame,scskey_list,bucket_dwalker)
bk_shop_weeks_buket_df = pd.concat(shop_weeks_buket)

<ipython-input-82-b30433df00a6>:29: RuntimeWarning: divide by zero encountered in longlong_scalars
  if a['sale_qty'] / total_sale_qty > 1:


## v2 버킷 디워커

In [113]:
# OW 
scskey_list = ['X20SDXSHA5011OW230','X20SDXSHA5011OW235','X20SDXSHA5011OW240','X20SDXSHA5011OW260','X20SDXSHA5011OW265','X20SDXSHA5011OW270']
frame = bucket_dwalker_v2[(bucket_dwalker_v2['scskey']=='X20SDXSHA5011OW240')][['shopkey','weekkey','week_takeout_qty','takeout_qty','week_sale_qty','sale_qty','stock_qty']].groupby('weekkey').sum()
bucket_dwalker_v2_shop_key = bucket_dwalker_v2['shopkey'].unique()


shop_weeks_buket_v2 = zero_fill(bucket_dwalker_v2_shop_key, frame,scskey_list,bucket_dwalker_v2)
ow_shop_weeks_buket_v2_df = pd.concat(shop_weeks_buket_v2)

In [98]:
# wh
scskey_list = ['X20SDXSHA1011WH230','X20SDXSHA1011WH235','X20SDXSHA1011WH240','X20SDXSHA1011WH260','X20SDXSHA1011WH265','X20SDXSHA1011WH270']
frame = bucket_dwalker_v2[(bucket_dwalker_v2['scskey']=='X20SDXSHA1011WH240')][['shopkey','weekkey','week_takeout_qty','takeout_qty','week_sale_qty','sale_qty','stock_qty']].groupby('weekkey').sum()
bucket_dwalker_v2_shop_key = bucket_dwalker_v2['shopkey'].unique()


shop_weeks_buket_v2 = zero_fill(bucket_dwalker_v2_shop_key, frame,scskey_list,bucket_dwalker_v2)
wh_shop_weeks_buket_v2_df = pd.concat(shop_weeks_buket_v2)

<ipython-input-82-b30433df00a6>:29: RuntimeWarning: divide by zero encountered in longlong_scalars
  if a['sale_qty'] / total_sale_qty > 1:
<ipython-input-82-b30433df00a6>:32: RuntimeWarning: divide by zero encountered in longlong_scalars
  frame.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] , a['sale_qty'] / total_sale_qty]


In [99]:
# BE
scskey_list = ['X20SDXSHA3011BE230','X20SDXSHA3011BE235','X20SDXSHA3011BE240','X20SDXSHA3011BE260','X20SDXSHA3011BE265','X20SDXSHA3011BE270']
frame = bucket_dwalker_v2[(bucket_dwalker_v2['scskey']=='X20SDXSHA3011BE240')][['shopkey','weekkey','week_takeout_qty','takeout_qty','week_sale_qty','sale_qty','stock_qty']].groupby('weekkey').sum()
bucket_dwalker_v2_shop_key = bucket_dwalker_v2['shopkey'].unique()


shop_weeks_buket_v2 = zero_fill(bucket_dwalker_v2_shop_key, frame,scskey_list,bucket_dwalker_v2)
be_shop_weeks_buket_v2_df = pd.concat(shop_weeks_buket_v2)

<ipython-input-82-b30433df00a6>:29: RuntimeWarning: divide by zero encountered in longlong_scalars
  if a['sale_qty'] / total_sale_qty > 1:
<ipython-input-82-b30433df00a6>:32: RuntimeWarning: divide by zero encountered in longlong_scalars
  frame.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] , a['sale_qty'] / total_sale_qty]


In [100]:
be_shop_weeks_buket_v2_df

,weekkey,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,stock_qty,shopkey,ratio,scskey,shop_nm,shop_nm_sfdc,repr_shopcode,type_shop_nm,type_contract_nm,type_zone_nm,region_nm
0,2019-11-18,1,1,0,0,1,X10052,0.00000,X20SDXSHA3011BE230,롯데강남,디스커버리 롯데 강남점,XH0254,일반매장,백화점특정,백화점,서울
1,2019-11-25,0,1,0,0,1,X10052,0.00000,X20SDXSHA3011BE230,롯데강남,디스커버리 롯데 강남점,XH0254,일반매장,백화점특정,백화점,서울
2,2019-12-02,0,1,0,0,1,X10052,0.00000,X20SDXSHA3011BE230,롯데강남,디스커버리 롯데 강남점,XH0254,일반매장,백화점특정,백화점,서울
3,2019-12-09,0,1,0,0,1,X10052,0.00000,X20SDXSHA3011BE230,롯데강남,디스커버리 롯데 강남점,XH0254,일반매장,백화점특정,백화점,서울
4,2019-12-16,4,5,2,2,3,X10052,0.20000,X20SDXSHA3011BE230,롯데강남,디스커버리 롯데 강남점,XH0254,일반매장,백화점특정,백화점,서울
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54,2020-11-30,0,11,0,8,3,X50096,0.88889,X20SDXSHA3011BE270,LF스퀘어양주(대-위),디스커버리 LF스퀘어 양주점,XH0084,일반매장,대리점,대리점,경기
55,2020-12-07,0,11,0,8,3,X50096,0.88889,X20SDXSHA3011BE270,LF스퀘어양주(대-위),디스커버리 LF스퀘어 양주점,XH0084,일반매장,대리점,대리점,경기
56,2020-12-14,0,11,1,9,2,X50096,1.00000,X20SDXSHA3011BE270,LF스퀘어양주(대-위),디스커버리 LF스퀘어 양주점,XH0084,일반매장,대리점,대리점,경기
57,2020-12-21,0,11,0,9,2,X50096,1.00000,X20SDXSHA3011BE270,LF스퀘어양주(대-위),디스커버리 LF스퀘어 양주점,XH0084,일반매장,대리점,대리점,경기


In [117]:
# buket
be_shop_weeks_buket_df.to_csv('be_shop_weeks_buket_df.csv')
wh_shop_weeks_buket_df.to_csv('wh_shop_weeks_buket_df.csv')
bk_shop_weeks_buket_df.to_csv('bk_shop_weeks_buket_df.csv')
gr_shop_weeks_buket_df.to_csv('gr_shop_weeks_buket_df.csv')

# v2
ow_shop_weeks_buket_v2_df.to_csv('ow_shop_weeks_buket_v2_df.csv')
wh_shop_weeks_buket_v2_df.to_csv('wh_shop_weeks_buket_v2_df.csv')
be_shop_weeks_buket_v2_df.to_csv('be_shop_weeks_buket_v2_df.csv')

# 실재고에 따른 개수 비교

## 버킷디워커

In [196]:
be_shop_weeks_buket_df_zero = shop_weeks_buket_function(be_shop_weeks_buket_df)
wh_shop_weeks_buket_df_zero = shop_weeks_buket_function(wh_shop_weeks_buket_df)
bk_shop_weeks_buket_df_zero = shop_weeks_buket_function(bk_shop_weeks_buket_df)
gr_shop_weeks_buket_df_zero = shop_weeks_buket_function(gr_shop_weeks_buket_df)

<ipython-input-102-f935b6beff54>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniqe_shop_scskey['amount_2ewm_week'] = ewm_value[:-1]
<ipython-input-102-f935b6beff54>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniqe_shop_scskey['qty'] = ewm_value[:-1]
<ipython-input-102-f935b6beff54>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org

In [198]:
# 현재 재고가 0 인 지점 개수 
print(len(be_shop_weeks_buket_df_zero))
print(len(wh_shop_weeks_buket_df_zero))
print(len(bk_shop_weeks_buket_df_zero))
print(len(gr_shop_weeks_buket_df_zero))

2225
9937
4925
903


In [218]:
bk_be_zero_count = bk_stock_zero_function(be_shop_weeks_buket_df_zero)
bk_wh_zero_count = bk_stock_zero_function(wh_shop_weeks_buket_df_zero)
bk_bk_zero_count = bk_stock_zero_function(bk_shop_weeks_buket_df_zero)
bk_gr_zero_count = bk_stock_zero_function(gr_shop_weeks_buket_df_zero)

In [200]:
# 로직을 사용했을 때 실재고 반영한 재고가 0 개인 지점 개수 
print(bk_be_zero_count)
print(bk_wh_zero_count)
print(bk_bk_zero_count)
print(bk_gr_zero_count)

916
9578
3878
638


## v2 버킷 디워커 

In [216]:
be_shop_weeks_v2_df_zero = shop_weeks_buket_v2_function(ow_shop_weeks_buket_v2_df)
wh_shop_weeks_v2_df_zero = shop_weeks_buket_v2_function(wh_shop_weeks_buket_v2_df)
ow_shop_weeks_v2_df_zero = shop_weeks_buket_v2_function(be_shop_weeks_buket_v2_df)


<ipython-input-212-4d93c3b2c745>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniqe_shop_scskey['amount_2ewm_week'] = ewm_value[:-1]
<ipython-input-212-4d93c3b2c745>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniqe_shop_scskey['qty'] = ewm_value[:-1]
<ipython-input-212-4d93c3b2c745>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org

In [217]:
# 현재 재고가 0 인 지점 개수 
print(len(be_shop_weeks_v2_df_zero))
print(len(wh_shop_weeks_v2_df_zero))
print(len(ow_shop_weeks_v2_df_zero))

240
244
325


In [219]:
v2_be_zero_count = v2_stock_zero_function(be_shop_weeks_v2_df_zero)
v2_wh_zero_count = v2_stock_zero_function(wh_shop_weeks_v2_df_zero)
v2_bk_zero_count = v2_stock_zero_function(ow_shop_weeks_v2_df_zero)

In [221]:
# 로직을 사용했을 때 실재고 반영한 재고가 0 개인 지점 개수 
print(v2_be_zero_count)
print(v2_wh_zero_count)
print(v2_bk_zero_count)


190
202
272
